In [1]:
import skimage
import numpy as np
from skimage import io, transform
import os
import shutil
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import urllib.request
import urllib.error

%matplotlib inline

In [15]:
root = "~/data"

dataset = ["real", "sim"]

imagePath = os.path.join(root, 'images')
labelsPath = os.path.join(root, 'labels')
linksPath = os.path.join(imagePath, 'imageLinks')
trainPath = os.path.join(imagePath, 'train')
testPath = os.path.join(imagePath, 'test')

print(root)

~/data


In [6]:
# Modified From:
# https://github.comr/datitran/raccoon_dataset/blob/master/xml_to_csv.py
import os

os.chdir(root)

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

for i in [os.path.join(root, 'train')]:
    image_path = i
    folder = os.path.basename(os.path.normpath(i))
    xml_df = xml_to_csv(image_path)
    
    from sklearn.model_selection import train_test_split
    
    train, test = train_test_split(xml_df, test_size=0.2)
    train.to_csv('data/train.csv', index=None)
    test.to_csv('data/test.csv', index=None)

In [7]:
!pwd

/home/bibagimon/CarND-Capstone/detection


In [12]:
!ls -lR

.:
total 36
drwxrwxr-x 2 bibagimon bibagimon 4096  4月 13 15:27 data
-rw-rw-r-- 1 bibagimon bibagimon 5330  4月 13 15:27 detection.ipynb
-rw-rw-r-- 1 bibagimon bibagimon   98  4月 10 22:46 docker-compose.yml
-rw-rw-r-- 1 bibagimon bibagimon 1110  4月 13 09:49 Dockerfile
-rw-rw-r-- 1 bibagimon bibagimon 3037  4月 13 15:18 generate_tf_record.py
-rw-rw-r-- 1 bibagimon bibagimon  184  4月 13 10:03 requirements.txt
-rw-rw-r-- 1 bibagimon bibagimon  209  4月 13 10:18 run-docker-container.sh
drwxrwxr-x 2 bibagimon bibagimon 4096  4月 13 09:41 training

./data:
total 8
-rw-rw-r-- 1 bibagimon bibagimon 48  4月 13 15:26 test.csv
-rw-rw-r-- 1 bibagimon bibagimon  0  4月 13 15:24 test.record
-rw-rw-r-- 1 bibagimon bibagimon 48  4月 13 15:26 train.csv
-rw-rw-r-- 1 bibagimon bibagimon  0  4月 13 15:24 train.record

./training:
total 24
-rw-rw-r-- 1 bibagimon bibagimon  246  4月 13 05:41 create-tf-record.sh
-rw-rw-r-- 1 bibagimon bibagimon 3714  4月 13 06:16 faster_rnn_resnet101.config
-rw-rw-r-- 1 bibagimon bibag